### Libraries

In [ ]:
import pandas as pd
import numpy as np
import unicodedata

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

### Options & Data

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("/home/b4/Documents/VisualCodeStudio/spotify_project/data/tracks_dataframe.csv", index_col=0)

### Functions

In [ ]:
# Search song name in dataframe
def search_song(song, dataframe):
  nfkd_form = unicodedata.normalize('NFKD', song)
  only_ascii = nfkd_form.encode('ASCII', 'ignore')
  search = only_ascii.decode()
  return dataframe[dataframe['name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.contains(search, case=False)]


In [ ]:
# Return 6 recommended songs based on features' similarity
## Function update: Search for song name or spotify's song ID
def recomended(song):    

  try:
    try:
      song_index = df[df['id']==song].index[0]
    except:
      song_index = df[df['name'].str.contains(song, case=False)].index[0]
    index_dist = list(zip(indices[song_index], distancias[song_index]))
    print(f"\nSearch Song: {df.iloc[song_index]['name']} - {df.iloc[song_index]['artist_name']}")
    for x, y in index_dist[1:]:
      print(f"""
            Distance: {round(y, 2)}
            Track: {df.iloc[x]['track_number']}
            Name: {df.iloc[x]['name']}
            Album: {df.iloc[x]['album_name']}
            Artist: {df.iloc[x]['artist_name']}
            Key: {df.iloc[x]['key']}
            """)
  except:
    print('Hmm.. Song not on the list.')


## Code

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df[df['danceability'].isnull()]

In [ ]:
df[df['name']=='Meu Consolador']

In [ ]:
df.drop(index=1200, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.isnull().sum()

### Model

In [ ]:
features = df[['danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
               'track_pop', 'album_pop', 'artist_pop']]

In [ ]:
features.isnull().sum()

In [ ]:
corr = features.corr()
matrix = np.triu(corr)
plt.subplots(figsize=(20,8))
sns.heatmap(features.corr(), annot=True, mask=matrix, cmap='coolwarm')

In [ ]:
std = StandardScaler()

In [ ]:
features_std = std.fit_transform(features)
features_std_df = pd.DataFrame(features_std, columns=features.columns)

In [ ]:
features_std_df

In [ ]:
model = KMeans(5)
cluster = model.fit_predict(features_std_df)

In [ ]:
df['cluster'] = cluster

### Visualization

In [ ]:
df.groupby(by='cluster').median()

In [ ]:
df[df['cluster']==4].describe()

In [ ]:
sns.boxplot(data=df, x='cluster', y='danceability')
plt.show()

### Recommendation

In [ ]:
cluster

In [ ]:
features_std_df['cluster'] = cluster

In [ ]:
features_std_df

In [ ]:
model = NearestNeighbors(n_neighbors=7)

In [ ]:
model.fit(features_std_df)

In [ ]:
distancias, indices = model.kneighbors(features_std_df)

In [ ]:
indices

In [ ]:
distancias

In [ ]:
song_index = df[df['name']=='Andando Sobre as Águas - Ao vivo'].index[0]

In [ ]:
indices[song_index]

In [ ]:
distancias[song_index]

In [ ]:
index_dist = list(zip(indices[song_index], distancias[song_index]))

In [ ]:
index_dist

In [ ]:
teste = list(index_dist)

In [ ]:
teste

In [ ]:
for x, y in index_dist:
  print(f"""
            Distance: {round(y, 2)}
            Track: {df.iloc[x]['track_number']}
            Name:{df.iloc[x]['name']}
            Album: {df.iloc[x]['album_name']}
            Artist: {df.iloc[x]['artist_name']}
            Key: {df.iloc[x]['key']}
        """)

## Recommended

In [ ]:
search_song('', df)

In [ ]:
recomended('')